In [1]:
from importlib import reload
import load_data
reload(load_data)
from load_data import *

In [6]:
url = "https://datasets.imdbws.com/"
filenames = ["name.basics.tsv.gz",
             "title.akas.tsv.gz",
             "title.basics.tsv.gz",
             "title.crew.tsv.gz",
             "title.episode.tsv.gz",
             "title.principals.tsv.gz",
             "title.ratings.tsv.gz"
]

output_paths = download_file(url, filenames, "data/")

# Pierwszy rzut oka na dane

Pierwsza ramka:

In [ ]:
path_to_file = "data/name.basics.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)

In [ ]:
df.head()

In [8]:
df.shape

(13583187, 6)

In [9]:
df.columns

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')

Druga ramka:

In [3]:
path_to_file = "data/title.akas.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)

In [7]:
df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,<NA>,<NA>,original,<NA>,1
1,tt0000001,2,Carmencita,DE,<NA>,<NA>,literal title,0
2,tt0000001,3,Carmencita,US,<NA>,imdbDisplay,<NA>,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,<NA>,imdbDisplay,<NA>,0
4,tt0000001,5,Καρμενσίτα,GR,<NA>,imdbDisplay,<NA>,0


In [8]:
df.shape

(48657469, 8)

In [9]:
df.columns

Index(['titleId', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'],
      dtype='object')

In [10]:
# wydostajemy kandydatów na filmy z których odzyskamy kraje
# zgodnie z uwagą w PDFie od prowadzących
df_think = df.loc[:, ['titleId', 'title', 'region', 'isOriginalTitle']].query(
    "isOriginalTitle == 0"
).merge(
    df.loc[:, ['titleId', 'title', 'isOriginalTitle']].query("isOriginalTitle == 1"),
    on="titleId",
    how="inner",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y", "isOriginalTitle", "isOriginalTitle_y"],
    axis=1
)

In [11]:
df_think

,titleId,title,region
0,tt0000001,Carmencita,DE
1,tt0000001,Carmencita,US
10,tt0000002,Le clown et ses chiens,FR
15,tt0000003,Pauvre Pierrot,FR
25,tt0000004,Un bon bock,FR
...,...,...,...
37819473,tt9916724,Hay Que Ser Paciente,CO
37819475,tt9916754,Chico Albuquerque - Revelações,BR
37819476,tt9916756,Pretty Pretty Black Girl,US
37819491,tt9916764,38,US


In [12]:
# grupujemy po ID tytułu
# potem zliczamy ile było przypisanych regionów
# tam, gdzie był jeden, daje się zidentyfikować jednoznacznie
ambig_index = df_think.loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count().where(
    lambda region: region > 1
).dropna().index

df_with_known_countries = df_think.loc[~df_think.titleId.isin(list(ambig_index))]

In [13]:
df_with_known_countries

,titleId,title,region
10,tt0000002,Le clown et ses chiens,FR
15,tt0000003,Pauvre Pierrot,FR
25,tt0000004,Un bon bock,FR
40,tt0000006,Chinese Opium Den,US
47,tt0000007,Corbett and Courtney Before the Kinetograph,US
...,...,...,...
37819473,tt9916724,Hay Que Ser Paciente,CO
37819475,tt9916754,Chico Albuquerque - Revelações,BR
37819476,tt9916756,Pretty Pretty Black Girl,US
37819491,tt9916764,38,US


In [15]:
# utrata danych: 26.6%
1 - df_with_known_countries.shape[0]/df_think.shape[0]

0.2663587633217803

In [18]:
# Teraz do każdego tytułu w 'df' przypisaliśmy kraj, jeśli się da
# jeśli nie, mamy 'NaN' w kolumnie 'region'
df = df.loc[df.isOriginalTitle == 1].loc[:, ["titleId", "title"]].merge(
            df_with_known_countries.loc[:, ["titleId", "region"]],
            how="left",
            on="titleId")

Trzecia ramka:

In [39]:
path_to_file = "data/title.basics.tsv.gz"
df = load_to_dataframe(path_to_file)

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:22: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [40]:
df.replace(to_replace=r"\N", value=pd.NA).head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"


Czwarta ramka:

In [ ]:
path_to_file = "data/title.crew.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()

Piąta ramka:

In [ ]:
path_to_file = "data/title.episode.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()

Szósta ramka:

In [ ]:
path_to_file = "data/title.principals.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()

Siódma ramka:

In [21]:
path_to_file = "data/title.ratings.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2058
1,tt0000002,5.6,277
2,tt0000003,6.5,2024
3,tt0000004,5.4,179
4,tt0000005,6.2,2791


# Task 1
Quality of movies by country

In [20]:
df_countries = df.copy()

In [26]:
df = df_countries.merge(
        df,
        how="left",
        left_on="titleId",
        right_on="tconst"
).drop(columns=["tconst"]).dropna()

,titleId,title,region,averageRating,numVotes
0,tt0000001,Carmencita,NaN,5.7,2058.0
1,tt0000002,Le clown et ses chiens,FR,5.6,277.0
2,tt0000003,Pauvre Pierrot,FR,6.5,2024.0
3,tt0000004,Un bon bock,FR,5.4,179.0
4,tt0000005,Blacksmith Scene,NaN,6.2,2791.0
...,...,...,...,...,...
10837713,tt9916848,Episode #3.17,NaN,NaN,NaN
10837714,tt9916850,Episode #3.19,NaN,NaN,NaN
10837715,tt9916852,Episode #3.20,NaN,NaN,NaN
10837716,tt9916856,The Wind,DE,NaN,NaN
